In [1]:
import copy
import time

def run_test(cases, fun, should_fail = False):
    print(f"Testing '{fun.__name__}'")
    for args, expected in cases:
        
        fun_args = copy.deepcopy(args)
        start = time.time()
        result = fun(*fun_args)
        end = time.time()
        time_str = '%5.2f' % ((end - start) * 10**6)
        
        args_str = ",".join([f"'{arg}'" for arg in args])
        if result == expected:
            print(f"\tOK {time_str}ns {args_str[:36] + (args_str[36:] and '..')}")
        elif should_fail:           
            assert expected == result, f"FAIL {args_str}. Expected: '{expected}'. Got: '{result}'"
        else:
            print(f"\tFAIL {args_str}. Expected: '{expected}'. Got: '{result}'")

In [2]:
class DualNode:
    def __init__(self, value, prev = None, next = None):
        self.value = value
        self.prev = prev
        self.next = next
        
    def __str__(self):
        return f"N({self.value})"
        
class LinkedList:
    def __init__(self):
        self.head = None
    
    def append(self, node):
        if self.head == None:
            self.head = node
            return self
        
        cur = self.head
        
        while cur.next != None:
            cur = cur.next
            
        cur.next = node
        node.prev = cur
        
        return self
    
    def append_many(self, nodes):
        for i in range(len(nodes)):
            if i != len(nodes) - 1:
                nodes[i].next = nodes[i + 1]
            if i != 0:
                nodes[i].prev = nodes[i - 1]
        self.append(nodes[0])
        
        return self
        
    def clear(self):
        cur = self.head
        
        while cur != None:
            cur, cur.prev, cur.next = cur.next, None, None
    
    def __eq__(self, other):
        cur_self = self.head
        cur_other = other.head
        
        while cur_self != None and cur_other != None:
            if cur_self.value != cur_other.value:
                return False
            
            cur_self = cur_self.next
            cur_other = cur_other.next
            
        return cur_self == cur_other
        
    def __str__(self):
        nodes = []
        
        cur = self.head
        while cur != None:
            nodes.append(cur)
            cur = cur.next
            
        return "↔".join([str(n.value) for n in nodes])
    
def as_dual_list(values):
    assert len(values) > 0
    return LinkedList().append_many([DualNode(v) for v in values])


# 2.1 Remove Dups

Write code to remove duplicates from an unsorted linked list.

*FOLLOW UP:* How would you solve this problem if a temporary buffer is not allowed?

In [3]:
def remove_dups_quad(list: LinkedList):
    outer = list.head
    while outer != None:
        inner = outer.next
        
        while inner != None:
            next = inner.next
            if outer.value == inner.value:
                if inner.prev != None:
                    inner.prev.next = inner.next
                if inner.next != None:
                    inner.next.prev = inner.prev
                inner.prev, inner.next = None, None
            inner = next
        outer = outer.next
    return list

def remove_dups_hash(list):
    cur = list.head
    values = set()
    while cur != None:
        next = cur.next

        if cur.value in values:
            cur.prev.next = next
            if next != None:
                next.prev = cur.prev
            cur.prev, cur.next = None, None
        else:
            values.add(cur.value)
        cur = next

    return list

In [4]:
test_cases = [
    ([as_dual_list([1, 2, 3])], as_dual_list([1, 2, 3])),
    ([as_dual_list([12, 11, 13, 14, 11, 30])], as_dual_list([12, 11, 13, 14, 30])),
    ([as_dual_list([1, 1, 1])], as_dual_list([1])),
    ([as_dual_list([1])], as_dual_list([1])),
    ([as_dual_list([1] * 300)], as_dual_list([1]))
]

run_test(test_cases, remove_dups_quad)
run_test(test_cases, remove_dups_hash)

Testing 'remove_dups_quad'
	OK  4.29ns '1↔2↔3'
	OK  6.68ns '12↔11↔13↔14↔11↔30'
	OK  3.81ns '1↔1↔1'
	OK  1.91ns '1'
	OK 237.94ns '1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1..
Testing 'remove_dups_hash'
	OK  6.91ns '1↔2↔3'
	OK 14.54ns '12↔11↔13↔14↔11↔30'
	OK  7.87ns '1↔1↔1'
	OK  3.10ns '1'
	OK 181.67ns '1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1..


# 2.2 Return Kth to Last

Implement an algorithm to find the kth to last element of a singly linked list.

In [5]:
class SingleNode:
    def __init__(self, value, next = None):
        self.value = value
        self.next = next
        
    def __str__(self):
        nodes = []
        cur = self
        
        while cur != None:
            nodes.append(cur)
            cur = cur.next
            
        return "→".join(str(n.value) for n in nodes)
    
    def __eq__(self, other):
        if other == None:
            return False

        cur_self = self
        cur_other = other
        while cur_self != None and cur_other != None:
            if cur_self.value != cur_other.value:
                return False
            cur_self = cur_self.next
            cur_other = cur_other.next
            
        return cur_self == cur_other
    
    def __len__(self):
        res = 0
        cur = self
        while cur != None:
            cur = cur.next
            res += 1
        return res
        
def as_single_list(values):
    assert len(values) > 0
    nodes = [SingleNode(v) for v in values]
    
    for i in range(len(nodes) - 1):
        nodes[i].next = nodes[i + 1]
    
    return nodes[0]
        
def kth_last_two_pass(node, k):
    assert k > 0
    cur = node
    n = 0
    while cur != None:
        cur = cur.next
        n += 1
    
    if k > n:
        return node
    
    cur = node
    for i in range(n - k):
        cur = cur.next
    return cur

def kth_last_extra_mem(node, k):
    assert k > 0
    nodes = []
    cur = node
    while cur != None:
        nodes.append(cur)
        cur = cur.next
    
    n = len(nodes)
    if k > n:
        return node
    return nodes[n - k]

def kth_last_two_pointers(node, k):
    assert k > 0
    
    fast = node
    fast_ind = 0
    
    slow = node
    slow_ind = 0
    
    while fast != None:
        fast = fast.next
        fast_ind += 1
        
        if fast_ind - k > slow_ind:
            slow = slow.next
            
    return slow

def kth_last_recursive(node, k):
    assert k > 0
    
    def kth_inner(n, i):
        if i == 0:
            return n, 0
        
        if n.next == None:
            return n, i - 1
        
        res, j = kth_inner(n.next, i)
        
        if j == 0:
            return res, 0
        return n, j - 1
    
    return kth_inner(node, k)[0]

In [6]:
test_cases = [
    ([as_single_list([1, 2, 3, 4, 5]), 1], as_single_list([5])),
    ([as_single_list([1] * 200), 200], as_single_list([1] * 200)),
    ([as_single_list([1, 2, 3, 4, 5]), 7], as_single_list([1, 2, 3, 4, 5])),
]

funs = [kth_last_two_pass, kth_last_extra_mem, kth_last_two_pointers, kth_last_recursive]
for f in funs:
    run_test(test_cases, f)

Testing 'kth_last_two_pass'
	OK 10.25ns '1→2→3→4→5','1'
	OK 59.37ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1..
	OK 22.17ns '1→2→3→4→5','7'
Testing 'kth_last_extra_mem'
	OK  5.25ns '1→2→3→4→5','1'
	OK 70.33ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1..
	OK 20.03ns '1→2→3→4→5','7'
Testing 'kth_last_two_pointers'
	OK  4.05ns '1→2→3→4→5','1'
	OK 110.63ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1..
	OK 19.07ns '1→2→3→4→5','7'
Testing 'kth_last_recursive'
	OK  5.01ns '1→2→3→4→5','1'
	OK 94.65ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1..
	OK 21.70ns '1→2→3→4→5','7'


# 2.3 Delete Middle Node

Implement an algorithm to delete a node in the middle (i.e., any node but
the first and last node, not necessarily the exact middle) of a singly linked list, given only access to
that node.
```
EXAMPLE
Input: the node c from the linked list a - >b- >c - >d - >e- >f
Result: nothing is returned, but the new linked list looks like a - > b - > d - > e - > f
```

In [7]:
def delete_middle(list: SingleNode):
    slow = list
    
    if slow.next == None or slow.next.next == None:
        return list
    
    fast = list.next.next
    
    i = 0
    while fast != None:
        fast = fast.next
        i += 1
        if i % 2 == 0:
            slow = slow.next
            
    
    to_remove = slow.next
    slow.next = to_remove.next
    to_remove.next = None
    
    return list

In [8]:
large_data = [i for i in range(100)]
del large_data[len(large_data) // 2]

test_cases = [
    ([as_single_list([1, 2, 3, 4, 5])], as_single_list([1, 2, 4, 5])),
    ([as_single_list([1, 2, 3, 4, 5, 6])], as_single_list([1, 2, 3, 5, 6])),
    ([as_single_list([1])], as_single_list([1])),
    ([as_single_list([1, 2])], as_single_list([1, 2])),
    ([as_single_list([1, 2, 3])], as_single_list([1, 3])),
    ([as_single_list([i for i in range(100)])], as_single_list(large_data)),
]

run_test(test_cases, delete_middle)

Testing 'delete_middle'
	OK  9.06ns '1→2→3→4→5'
	OK  5.48ns '1→2→3→4→5→6'
	OK  1.91ns '1'
	OK  1.67ns '1→2'
	OK  3.58ns '1→2→3'
	OK 63.42ns '0→1→2→3→4→5→6→7→8→9→10→11→12→13→14→..


# 2.4 Partition

Write code to partition a linked list around a value x, such that all nodes less than x come
before all nodes greater than or equal to x. Ifxis contained within the list, the values of x only need
to be after the elements less than x (see below). The partition element x can appear anywhere in the
"right partition"; it does not need to appear between the left and right partitions.
```
EXAMPLE
Input: 3 -> 5 -> 8 -> 5 -> 10 -> 2 -> 1 [partition = 5]
Output: 3 -> 1 -> 2 -> 10 -> 5 -> 5 -> 8
```

In [9]:
def prev_links(list):
    cur = list.head
    while cur.next != None:
        cur = cur.next
        
    nodes = []
    while cur != None:
        nodes.append(cur)
        cur = cur.prev
        
    nodes = nodes[::-1]
    return '→'.join(str(n.value) for n in nodes)

def partition(list, pivot):
    tail = list.head
    n = 0
    while tail.next != None:
        tail = tail.next
        n += 1
        
    left = list.head
    right = tail
    i = 0
    j = n
    
    while i < j and left != None and right != None:
        while left.next != None and left.value < pivot and i < n:
            i += 1
            left = left.next
        
        while right.prev != None and right.value > pivot and j > 0:
            j -= 1
            right = right.prev

        if i >= j:
            return list
        
        
        if j - i == 1:
            # a - 18 - 13 - b
            # a - 13 - 18 - b
            if left.prev != None:
                left.prev.next = right
            
            if right.next != None:
                right.next.prev = left 
            
            right.next, left.prev, left.next, right.prev = left, right, right.next, left.prev
        else:
            if left.prev == None:
                list.head = right
                left.next.prev = right
            else:
                left.prev.next, left.next.prev = right, right

            if right.next == None:
                right.prev.next = left
            else:
                right.prev.next, right.next.prev = left, left

            right.prev, left.prev = left.prev, right.prev
            right.next, left.next = left.next, right.next

        right, left = left, right
    return list



In [10]:
test_cases = [
    ([as_dual_list([5, 4, 3, 2, 1]), 3], as_dual_list([1, 2, 3, 4, 5])),
    ([as_dual_list([1, 2, 3, 4, 5]), 3], as_dual_list([1, 2, 3, 4, 5])),
    ([as_dual_list([15, 12, 18, 10, 20, 13, 6, 28]), 18], as_dual_list([15, 12, 6, 10, 13, 18, 20, 28]))
]

run_test(test_cases, partition)


Testing 'partition'
	OK 16.21ns '5↔4↔3↔2↔1','3'
	OK  8.82ns '1↔2↔3↔4↔5','3'
	OK 17.88ns '15↔12↔18↔10↔20↔13↔6↔28','18'


# 2.5 Sum Lists
You have two numbers represented by a linked list, where each node contains a single
digit. The digits are stored in reverse order, such that the Vs digit is at the head of the list. Write a
function that adds the two numbers and returns the sum as a linked list.
```
EXAMPLE
Input: ( 7 - > 1 -> 6) + (5 -> 9 -> 2).That is,617 + 295.
Output: 2 -> 1 -> 9. That is, 912.
FOLLOW UP
Suppose the digits are stored in forward order. Repeat the above problem.
EXAMPLE
Input: (6 -> 1 -> 7) + (2 -> 9 -> 5).That is, 617 + 295,
Output:9 -> 1 -> 2. That is,912.
```

In [11]:
def sum_lists(a, b):
    cur_a = a
    cur_b = b
    overflow = 0
    
    head = None
    cur = head
    
    while cur_a != None and cur_b != None:
        node_sum = cur_a.value + cur_b.value + overflow
        
        if head == None:
            head = cur = SingleNode(node_sum % 10)
        else:
            cur.next = SingleNode(node_sum % 10)
            cur = cur.next

        overflow = node_sum // 10
        cur_a = cur_a.next
        cur_b = cur_b.next
    
    if cur_b != None:
        cur_a = cur_b
        
    while cur_a != None:
        node_sum = overflow + cur_a.value
        cur.next = SingleNode(node_sum % 10)
        cur = cur.next
        
        cur_a = cur_a.next
        overflow = node_sum // 10
    
    if overflow > 0:
        cur.next = SingleNode(overflow)
    
    return head

In [12]:
test_cases = [
    ([as_single_list([7,1,6]), as_single_list([5,9,2])], as_single_list([2, 1, 9])),
    ([as_single_list([9, 8, 7]), as_single_list([1, 1, 2])], as_single_list([0, 0, 0, 1])),
    ([as_single_list([1, 2]), as_single_list([1, 1, 2])], as_single_list([2, 3, 2])),
    ([as_single_list([1, 1, 2, 0, 1, 1, 0]), as_single_list([1, 2])], as_single_list([2, 3, 2, 0, 1, 1, 0])),
]

run_test(test_cases, sum_lists)


Testing 'sum_lists'
	OK 29.33ns '7→1→6','5→9→2'
	OK 21.70ns '9→8→7','1→1→2'
	OK 17.40ns '1→2','1→1→2'
	OK 321.63ns '1→1→2→0→1→1→0','1→2'


In [13]:
def sum_reversed(a, b):
    cur_a, cur_b = a,b
    
    len_a = 0
    while cur_a != None and cur_b != None:
        cur_a = cur_a.next
        cur_b = cur_b.next
        len_a += 1
        
    if cur_a == None:
        a, b = b, a
        cur_a, cur_b = cur_b, cur_a
    
    len_b = len_a
    
    while cur_a != None:
        cur_a = cur_a.next
        len_a += 1
        
    if len_a != len_b:
        nodes = [SingleNode(0) for n in range(len_a - len_b)]
        for i in range(len(nodes) - 1):
            nodes[i].next = nodes[i + 1]
        nodes[-1].next = b
        b = nodes[0]
    
    def rec_sum(a, b):
        if a.next == None:
            s = a.value + b.value
            return SingleNode(s % 10), s // 10
        
        node, over = rec_sum(a.next, b.next)
        s = a.value + b.value + over
        return SingleNode(s % 10, node), s // 10
        
    
    node, over = rec_sum(a, b)
    if over > 0:
        node = SingleNode(over, node)
    return node

In [14]:
test_cases = [
    ([as_single_list([9, 9]), as_single_list([9, 9])], as_single_list([1, 9, 8])),
    ([as_single_list([1, 2]), as_single_list([1, 1, 4, 6, 7, 9])], as_single_list([1, 1, 4, 6, 9, 1])),
]
run_test(test_cases, sum_reversed)

Testing 'sum_reversed'
	OK 11.92ns '9→9','9→9'
	OK 19.55ns '1→2','1→1→4→6→7→9'


# 2.6 Palindrome
Implement a function to check if a linked list is a palindrome.

In [15]:
def is_palindrome(list):
    def inner(node):
        if node.next == None:
            return node.value == list.value, list.next
        bound_eq, next_node = inner(node.next)
        if not bound_eq:
            return False, None
        return bound_eq and node.value == next_node.value, next_node.next
    return inner(list)[0]

In [16]:
test_cases = [
    ([as_single_list([1, 2, 3, 2, 1])], True),
    ([as_single_list([1, 2, 3, 4, 5])], False),
    ([as_single_list([1, 2, 1])], True),
    ([as_single_list([1, 1])], True),
    ([as_single_list([1])], True),
]


run_test(test_cases, is_palindrome)

Testing 'is_palindrome'
	OK  6.20ns '1→2→3→2→1'
	OK  3.81ns '1→2→3→4→5'
	OK  3.58ns '1→2→1'
	OK  3.10ns '1→1'
	OK  2.15ns '1'


# 2.7 Intersection

Given two (singly) linked lists, determine if the two lists intersect. Return the inter-
secting node. Note that the intersection is defined based on reference, not value. That is, if the kth
node of the first linked list is the exact same node (by reference) as the j t h node of the second
linked list, then they are intersecting.

In [30]:
def intersection_smart(a, b):
    m = len(a) # O(m)
    n = len(b) # O(n)
    if n > m:
        m, n = n, m
        a, b = b, a
        
    k = m - n
    
    cur_a, cur_b = a, b
    while k > 0:
        cur_a = cur_a.next
        k -=1 
        
    while cur_a != None:
        if id(cur_a) == id(cur_b):
            return True
        cur_a = cur_a.next
        cur_b = cur_b.next
        
    return False

def intersection_hash(a, b):
    visited = set()
    
    cur_a = a
    while cur_a != None:
        visited.add(id(cur_a))
        cur_a = cur_a.next
        
    cur_b = b
    while cur_b != None:
        if id(cur_b) in visited:
            return True
        cur_b = cur_b.next
        
    return False

In [32]:
test_cases = []

# case 1: same from middle
a = as_single_list([1, 2, 3, 4, 5])
b = a
c = 0
while c != 2:
    c += 1
    b = b.next
test_cases.append(([a, b], True))

# case 2: same last
a = as_single_list([1, 2, 3, 4, 5])
b = a
c = 0
while c != 4:
    c += 1
    b = b.next
test_cases.append(([a, b], True))

# case 3: links differ
a = as_single_list([1, 2, 3, 4, 5])
b = as_single_list([3, 4, 5])
test_cases.append(([a, b], False))



run_test(test_cases, intersection_smart)
run_test(test_cases, intersection_hash)

Testing 'intersection_smart'
	OK 20.98ns '1→2→3→4→5','3→4→5'
	OK  6.91ns '1→2→3→4→5','5'
	OK  7.63ns '1→2→3→4→5','3→4→5'
Testing 'intersection_hash'
	OK 10.01ns '1→2→3→4→5','3→4→5'
	OK 10.73ns '1→2→3→4→5','5'
	OK  7.15ns '1→2→3→4→5','3→4→5'


# 2.8 Loop Detection
Given a circular linked list, implement an algorithm that returns the node at the
beginning of the loop.
```
DEFINITION
Circular linked list: A (corrupt) linked list in which a node's next pointer points to an earlier node, so
as to make a loop in the linked list.
```
```
EXAMPLE
Input: A - > 8 - > C - > D - > E - > C [the same C as earlier]
Output: C
Hints: #50, #69, #83, #90
```

In [51]:
def has_loop_smart(a):
    slow, fast = a, a
    should_slow_move = False
    
    while fast != None:
        if should_slow_move:
            if id(slow) == id(fast):
                return True
            slow = slow.next
        fast = fast.next
        should_slow_move = not should_slow_move
        
    return False

def has_loop_hash(a):
    visited = set()
    
    while a != None:
        ref = id(a)
        if ref in visited:
            return True
        visited.add(ref)
        a = a.next
    return False

In [53]:
test_cases = []
# case 1: has loop in middle
a = as_single_list([1, 2, 3, 4, 5])

b = a
i = 0
while i < 2:
    i += 1
    b = b.next
c = b

while c.next != None:
    c = c.next
c.next = b

test_cases.append(('has loop in middle', a, True))

# case 1: has loop in middle
a = as_single_list([1, 2, 3, 4, 5])

b = a
i = 0
while i < 3:
    i += 1
    b = b.next
c = b

while c.next != None:
    c = c.next
c.next = b

test_cases.append(('has loop in middle + 1', a, True))

# case 3: no loop
a = as_single_list([1, 2, 3, 4, 5])
test_cases.append(('no loop', a, False))

fun = [has_loop_smart, has_loop_hash]
for f in fun:
    print(f"Testing '{f.__name__}'")
    for c in test_cases:
        res = has_loop_smart(c[1])
        if res == c[2]:
            print(f"\tOK '{c[0]}'")
        else:
            print(f"\tFAIL '{c[0]}'. Expected '{c[2]}'. Got '{res}'")

Testing 'has_loop_smart'
	OK 'has loop in middle'
	OK 'has loop in middle + 1'
	OK 'no loop'
Testing 'has_loop_hash'
	OK 'has loop in middle'
	OK 'has loop in middle + 1'
	OK 'no loop'
